<a href="https://colab.research.google.com/github/Nanou05/Covid-19-dashboard/blob/main/survival_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install lifelines # installer le package Lifelines qui nous permettra d'utiliser Kaplan Meier

     |████████████████████████████████| 348kB 6.6MB/s 
     |████████████████████████████████| 61kB 5.1MB/s 
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-cp37-none-any.whl size=4035 sha256=f327247b50d20a942230d9d045c902846cabc911cda56c018726b5a7e90c8541
  Stored in directory: /root/.cache/pip/wheels/dc/68/dc/91321c55fba449755524481854f5be70d41912b8f886f908bb
Successfully built autograd-gamma


In [1]:
# MOUNTING GOOGLE COLAB WITH THE DRIVE ACCOUNT

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
# Importer les packages

import pandas as pd
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
import io
import numpy as np

In [19]:
# télécharger les données mondiales
df1 = pd.read_csv('/content/drive/My Drive/latestdata.csv', sep = ',', error_bad_lines= False,parse_dates=False)
df1.head(5)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,9,10,12,13,14,15,16,17,19,20,21,22,23,24,25,26,27,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,age,sex,city,province,country,latitude,longitude,geo_resolution,date_onset_symptoms,date_admission_hospital,date_confirmation,symptoms,lives_in_Wuhan,travel_history_dates,travel_history_location,reported_market_exposure,additional_information,chronic_disease_binary,chronic_disease,source,sequence_available,outcome,date_death_or_discharge,notes_for_discussion,location,admin3,admin2,admin1,country_new,admin_id,data_moderator_initials,travel_history_binary
0,000-1-1,NaN,male,Shek Lei,Hong Kong,China,22.365019,114.133808,point,NaN,NaN,14.02.2020,NaN,NaN,22.01.2020,China,NaN,Case 55; mainland China travel via the Lok Ma ...,False,NaN,https://www.scmp.com/news/hong-kong/health-env...,NaN,"critical condition, intubated as of 14.02.2020",NaN,NaN,Shek Lei,NaN,NaN,Hong Kong,China,8029.0,NaN,NaN
1,000-1-10,78,male,Vo Euganeo,Veneto,Italy,45.297748,11.658382,point,NaN,NaN,21.02.2020,NaN,NaN,NaN,NaN,NaN,Hospitalized on 12.02.2020 for other reasons,False,NaN,https://www.corriere.it/cronache/20_febbraio_2...,NaN,death,22.02.2020,NaN,Vo' Euganeo,NaN,NaN,Veneto,Italy,8954.0,NaN,NaN
2,000-1-100,61,female,NaN,NaN,Singapore,1.353460,103.815100,admin0,NaN,NaN,14.02.2020,NaN,NaN,NaN,NaN,NaN,"Case 65; family member of Case 50, a DBS emplo...",False,NaN,https://www.channelnewsasia.com/news/singapore...,NaN,discharge,17.02.2020,NaN,NaN,NaN,NaN,NaN,Singapore,200.0,NaN,NaN
3,000-1-1000,NaN,NaN,Zhengzhou City,Henan,China,34.629310,113.468000,admin2,NaN,NaN,26.01.2020,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,https://news.163.com/special/epidemic/?spssid=...,NaN,NaN,NaN,NaN,NaN,NaN,Zhengzhou City,Henan,China,10091.0,NaN,NaN
4,000-1-10000,NaN,NaN,Pingxiang City,Jiangxi,China,27.513560,113.902900,admin2,NaN,NaN,14.02.2020,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,http://hc.jiangxi.gov.cn/doc/2020/02/15/138898...,NaN,NaN,NaN,NaN,NaN,NaN,Pingxiang City,Jiangxi,China,7060.0,NaN,NaN


In [21]:
# garder seulement les variables intéressantes pour notre analyse:

df= df1[['date_onset_symptoms','date_confirmation','date_death_or_discharge','date_admission_hospital','outcome','sex','age','country']]


In [23]:
#renommer les colonnes des données

df = df.rename(columns={"date_onset_symptoms": "onset", "date_admission_hospital": "admission", "date_confirmation" : "confirmation"
                     , "date_death_or_discharge" : "death_discharge"})
# indexing rows with no missing values
index_onset = ~df['onset'].isna()
index_admission = ~df['admission'].isna()
index_confirmation = ~df['confirmation'].isna()
index_death_discharge = ~df['death_discharge'].isna()

In [24]:
# enlever les lignes où ces données sont manquantes simultanément:

a = index_onset | index_admission  | index_death_discharge
df = df[a]
df= df.reset_index()
df =  df.drop(['index'], axis=1)

In [27]:
#unifier les nomes des outcomes:

for i in range(np.shape(df)[0]):
    if df['outcome'][i] == 'discharged':
        df['outcome'][i] = 'discharge'
    elif df['outcome'][i] == 'Discharged':
        df['outcome'][i] = 'discharge'
    elif df['outcome'][i] == 'Death':
        df['outcome'][i] = 'death'
    elif df['outcome'][i] == 'died':
        df['outcome'][i] = 'death'
    elif df['outcome'][i] == 'dead':
        df['outcome'][i] = 'death'
    elif df['outcome'][i] == 'released from quarantine':
        df['outcome'][i] = 'discharge'
    elif df['outcome'][i] == 'stable':
        df['outcome'][i] = 'discharge'
    elif df['outcome'][i] == 'recovered':
        df['outcome'][i] = 'discharge'
    elif df['outcome'][i] == 'severe':
        df['outcome'][i] = 'death'
    elif df['outcome'][i] == 'severe':
        df['outcome'][i] = 'death'

In [29]:
#enlever les outcomes inconnus
df = df.drop([79,614])

In [30]:
#Cette fois on analyse seulement les décharges (sorties) donc on enlève les décès:

a = df['outcome'] == 'death'
a = ~a
df = df[a]
df = df.reset_index()
df =  df.drop(['index'], axis=1)

In [32]:
#faire des assumptions sur les dates:

for i in range(np.shape(df)[0]):
    if index_onset[i] == False and index_admission[i] == True:
        df['onset'][i] = df['admission'][i]
    elif index_onset[i] == False and index_admission[i] == False and index_confirmation[i] == True:
        df['onset'][i] = df['confirmation'][i]

In [34]:
#reformatter les dates:

df['onset'] = df['onset'].apply(pd.to_datetime, dayfirst=True)
df['confirmation'] = df['confirmation'].apply(pd.to_datetime, dayfirst=True)
df['death_discharge'] = df['death_discharge'].apply(pd.to_datetime, dayfirst=True)

In [ ]:
#le status de censure:
## NB: ce code peut mettre bcp de temps pour se lancer, il vaut mieux utiliser une machine puissante ou lancer le code sur google colab
index_censoring = ~df['death_discharge'].isna()
#calculer les evénements et le nombre de jours:

event_censoring_days = []
for i in range(np.shape(df)[0]):
    if index_censoring[i] == False:
        event_censoring_days = np.append(event_censoring_days,df['confirmation'].dt.dayofyear[i] - df['onset'].dt.dayofyear[i] )
    elif index_censoring[i] == True:
        event_censoring_days = np.append(event_censoring_days, df['death_discharge'].dt.dayofyear[i] - df['onset'].dt.dayofyear[i])
# à cause de certaines inconsistances, quelques jours-évenements censurés sont négatifs, nous les enlevons et ajustons les données
a = event_censoring_days > 0
event_censoring_days = event_censoring_days[a]
df = data[a]
data = df.reset_index()
df =  df.drop(['index'], axis=1)
index_censoring = index_censoring[a]


In [ ]:
inddaypos = event_censoring_days > 0
event_censoring_days = event_censoring_days[inddaypos]
df = df[inddaypos]
df = df.reset_index()
df = df.drop(['index'], axis=1)

index_censoring = index_censoring[inddaypos]

In [ ]:
frm = {'event_censoring_days': event_censoring_days, 'status': index_censoring}
data_y_unstructured = pd.DataFrame(data=frm)
data_y_unstructured = data_y_unstructured[['status', 'event_censoring_days']]
s = data_y_unstructured.dtypes
data_y = np.array([tuple(x) for x in data_y_unstructured.values], dtype=list(zip(s.index, s)))

In [40]:
gender = df['sex']
gender = gender.astype('category')

In [ ]:
# estimer la médiane de l'age et la visualiser:

median = df['age'].median()
#age_type = 'categorized'
age_type = 'median'
if age_type == 'median':
    for i in range(len(df['age'])):
        if df['age'][i] > median:
            df['age'][i] = 'age > 46'
        else:
            df['age'][i] = 'age < 46'
    for value in df['age'].unique():
        mask = df['age'] == value
        time, survival_prob =KaplanMeierFitter(data_y["status"][mask],data_y["event_censoring_days"][mask])
        survival_prob = 1 - survival_prob
        plt.step(time, survival_prob, where="post",
                 label="%s" % (value))
    plt.ylabel("est. probability of survival $\hat{S}(t)$")
    plt.xlabel("time $t$")
    plt.legend(loc="best")
    limit = plt.gca()
    limit.set_xlim([0,45])
    limit.set_ylim([0, 1])
    plt.savefig('covid_median_age.pdf')

In [ ]:
# Visualiser les estimations par rapport à l'âge:

if age_type == 'categorized':
    for i in range(len(df['age'])):
        if df['age'][i] > 60:
            df['age'][i] = 'age > 60'
        elif df['age'][i] <= 60 and df['age'][i] > 46:
            df['age'][i] = '46 < age < 60'
        elif df['age'][i] <= 46 and df['age'][i] > 34:
            df['age'][i] = '34 < age < 46'
        else:
            df['age'][i] = 'age < 34'
    for value in df['age'].unique():
        mask = df['age'] == value
        time, survival_prob = KaplanMeierFitter(data_y["status"][mask],data_y["event_censoring_days"][mask])
        survival_prob = 1 - survival_prob
        plt.step(time, survival_prob, where="post",
                 label="%s" % (value))

    plt.ylabel("est. probability of survival $\hat{S}(t)$")
    plt.xlabel("time $t$")
    plt.legend(loc="best")
    limit = plt.gca()
    limit.set_xlim([0,37])
    limit.set_ylim([0, 1])
    plt.savefig('covid_age_categorized.pdf')

In [ ]:
# visualiser les estimations par rapport au sexe:

for value in gender.unique():
    mask = gender == value
    if value == 'male':
        time_cell, survival_prob_cell = KaplanMeierFitter(data_y["status"][mask],data_y["event_censoring_days"][mask])
        plt.step(time_cell, 1 - survival_prob_cell, where="post", color = 'r',label="%s" % (value))
    else:
        time_cell, survival_prob_cell = KaplanMeierFitter(data_y["status"][mask],data_y["event_censoring_days"][mask])
        plt.step(time_cell, 1 - survival_prob_cell, where="post",color = 'b',label="%s" % (value))
plt.ylabel("est. probability of discharge $\hat{S}(t)$")
plt.xlabel("time $t$")
plt.legend(loc="best")
limit = plt.gca()
limit.set_xlim([0, 42])
limit.set_ylim([0, 1])
plt.savefig('corona_gender_onset.pdf')